In [ ]:
#Final Arima+
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mysql.connector
import dotenv
from dotenv import dotenv_values
import os
import pandas as pd


#reding the table from server-----------------------------------------------------------------------------------------------------------------

filename = os.path.basename(__file__)
path = str(__file__).replace(filename, '')
env = dotenv_values(path + '.env')
# env = dotenv_values(r'C:\Users\satia\Desktop\137\zobale'+'.env')
query = "SELECT * "
query += " FROM `request_statistics`"
query += " "

try:
    connection = mysql.connector.connect(host=env['DB_HOST'], database=env['DB_DATABASE'], user=env['DB_USERNAME'],
                                         password=env['DB_PASSWORD'])

#    connection = mysql.connector.connect(host="192.168.168.125", database="requestdb", user="requestuser",
                                         #password="SxfWL7myDid1JBqS")
except mysql.connector.Error as err:
    print('There was an error connecting to the database!!!')
    print(err)
    exit()
cursor = connection.cursor(dictionary=True)
cursor.execute(query)
last = cursor.fetchall()
last = pd.DataFrame(last)
# print (dfm)
if connection.is_connected():
    cursor.close()
    connection.close()

#removing unnecessary columns of the table--------------------------------------------------------------------------------------------------------

last = last.drop('statisticsType', axis=1)
last = last.drop('subjectID', axis=1)
last = last.drop('id', axis=1)
last = last.drop('created_at', axis=1)
last = last.drop('updated_at', axis=1)
last.drop(last.tail(1).index,inplace=True)
#_______________________________________________________________________________________________________
last = last.drop('time', axis=1)
print (last)

data=last.copy()
# Function to handle outliers/anomalies
def iqr_fence(x):
    Q1 = x.quantile(0.25)
    Q3 = x.quantile(0.75)
    IQR = Q3 - Q1
    Lower_Fence = Q1 - (1.5 * IQR)
    Upper_Fence = Q3 + (1.5* IQR)
    u = max(x[x<Upper_Fence])
    l = min(x[x>Lower_Fence])
    return [u,l]


def handle_outliers(data):

    for column in data.columns:
        threshold=iqr_fence(data[column])[0]
        data[column] = data[column].mask(data[column] > threshold, int(threshold))
    return data

# Handle outliers/anomalies in the dataset
data = handle_outliers(data)
print(data)

# Define the objective function for hyperopt
def arima_objective(params):
    p, d, q = params['p'], params['d'], params['q']

    try:
        # Fit ARIMA model
        model = ARIMA(train_data, order=(p, d, q))
        model_fit = model.fit()

        # Make one-step ahead predictions
        predictions = model_fit.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1, typ='levels')

        # Calculate MSE
        mse = mean_squared_error(test_data, predictions)

        return {'loss': mse, 'status': STATUS_OK}

    except:
        return {'loss': np.inf, 'status': STATUS_OK}
Prediction_Metrice=[]
MAE_Percentage_Error=[]
# Iterate over each region column
for region in data.columns:
    # Get the region data
    region_data = data[region].values

    # Split the data into train and test sets
    train_size = int(len(region_data) * 0.8)
    train_data, test_data = region_data[:train_size], region_data[train_size:]

    # Define the search space for hyperopt
    space = {
        'p': hp.choice('p', range(0, 5)),
        'd': hp.choice('d', range(0, 5)),
        'q': hp.choice('q', range(0, 5))
    }

    # Run hyperopt optimization
    trials = Trials()
    best = fmin(fn=arima_objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

    # Get the best parameters
    best_p = best['p']
    best_d = best['d']
    best_q = best['q']

    # Fit ARIMA model with best parameters
    best_model = ARIMA(train_data, order=(best_p, best_d, best_q))
    best_model_fit = best_model.fit()

    # Make final predictions with the best model
    predictions = best_model_fit.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1, typ='levels')

    Real_prediction = best_model_fit.predict(start=len(train_data) + len(test_data)+1 , end=len(train_data) + len(test_data)+1 , typ='levels')

    Prediction_Metrice.append(Real_prediction)



        # Calculate evaluation metrics
    mse = mean_squared_error(test_data, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_data, predictions)
    r2 = r2_score(test_data, predictions)


    MAE_Percentage_Error.append(mae*100/np.mean(data[region][-4:]))

    # Print the evaluation metrics
    print("Region:", region)
    print("Best Parameters: p =", best_p, "d =", best_d, "q =", best_q)
    print("Test Actual values:", test_data)
    print("Test Predicted values:", predictions)
    print("One Step Ahead Value:", Real_prediction)
    print("MSE:", mse)
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R2:", r2)
    print()

print("Prediction_Metrice:", Prediction_Metrice)
print("MAE_Percentage_Error:", MAE_Percentage_Error)